In [2]:
!pip install transformers

     |████████████████████████████████| 1.4MB 5.5MB/s 
     |████████████████████████████████| 890kB 22.7MB/s 
     |████████████████████████████████| 2.9MB 31.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=eaf3c087a9643e258ebf91bfed4c5a70f52642baefabe2f8a35b6970641891c0
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
from collections import OrderedDict

In [4]:
from google.colab import files
uploaded = files.upload()

Saving dev-v2.0.json to dev-v2.0.json


In [5]:
import json

In [6]:
def read_file(file_name):
  with open(file_name) as f:
    data = json.load(f)
  contexts = list()
  questions = list()
  answers = list()
    
  for group in data['data']:
      for passage in group['paragraphs']:
          context = passage['context']
          for qa in passage['qas']:
              question = qa['question']
              an = []
              if(qa['is_impossible']):
                an.append('')
              else:
                for answer in qa['answers']:
                  an.append(answer['text'])
              contexts.append(context)
              questions.append(question)
              answers.append(an)

  return contexts, questions, answers

In [7]:
context, question, answer = read_file('dev-v2.0.json')

In [8]:
context[0:10]

['The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.',
 'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates 

In [9]:
len(context)

11873

In [10]:
question[0:10]

['In what country is Normandy located?',
 'When were the Normans in Normandy?',
 'From which countries did the Norse originate?',
 'Who was the Norse leader?',
 'What century did the Normans first gain their separate identity?',
 "Who gave their name to Normandy in the 1000's and 1100's",
 'What is France a region of?',
 'Who did King Charles III swear fealty to?',
 'When did the Frankish identity emerge?',
 'Who was the duke in the battle of Hastings?']

In [12]:
answer[0:10]

[['France', 'France', 'France', 'France'],
 ['10th and 11th centuries',
  'in the 10th and 11th centuries',
  '10th and 11th centuries',
  '10th and 11th centuries'],
 ['Denmark, Iceland and Norway',
  'Denmark, Iceland and Norway',
  'Denmark, Iceland and Norway',
  'Denmark, Iceland and Norway'],
 ['Rollo', 'Rollo', 'Rollo', 'Rollo'],
 ['10th century', 'the first half of the 10th century', '10th', '10th'],
 [''],
 [''],
 [''],
 [''],
 ['William the Conqueror', 'William the Conqueror', 'William the Conqueror']]

In [13]:
#Model
bert_model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

#Tokenizer
bert_tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [14]:
def get_answers(context, question, model, tokenizer):
  answers = []
  for c, q in zip(context, question):
    encoding = tokenizer.encode_plus(q,c, return_tensors='pt')
    inputs = encoding['token_type_ids'].lt(1)  #Token embeddings

    qt = torch.masked_select(encoding['input_ids'], inputs)

    chunk_size = model.config.max_position_embeddings - qt.size()[0] - 1

    chunked_input = OrderedDict()
    for k,v in encoding.items():
      q = torch.masked_select(v, inputs)
      c = torch.masked_select(v, ~inputs)
      chunks = torch.split(c, chunk_size)

      for i, chunk in enumerate(chunks):
        if i not in chunked_input:
          chunked_input[i] = {}

        thing = torch.cat((q, chunk))
        if i != len(chunks)-1:
          if k == 'input_ids':
            thing = torch.cat((thing, torch.tensor([102])))
          else:
            thing = torch.cat((thing, torch.tensor([1])))

        chunked_input[i][k] = torch.unsqueeze(thing, dim=0)
 
    def convert_ids_to_string(tokenizer, input_ids):
      return tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids))
    answer = ''

    # iterate over our chunks to look for the best answer from each chunk
    for _, chunk in chunked_input.items():
        m = model(**chunk)
        answer_start_scores = m[0]
        answer_end_scores = m[1]

        answer_start = torch.argmax(answer_start_scores)
        answer_end = torch.argmax(answer_end_scores) + 1

        ans = convert_ids_to_string(tokenizer, chunk['input_ids'][0][answer_start:answer_end])
        
        # if the ans == [CLS] then the model did not find a real answer in this chunk
        if ans != '[CLS]':
            answer += ans + " "
#    answers.append({'answer_start': start_index, 'text': corrected_answer})
    answers.append(answer.strip())
  return answers

In [15]:
bert_answer = get_answers(context[0:500], question[0:500], bert_model, bert_tokenizer)

In [40]:
#import pickle

#with open('bert_answer_pickle', 'wb') as bp:
#    pickle.dump(bert_answer, bp)

In [ ]:
#with open ('bert_answer_pickle', 'rb') as bp:
#    bert_answer = pickle.load(bp)

In [16]:
len(bert_answer)

500

In [27]:
#print(bert_answer)

['france', '10th and 11th centuries', 'denmark , iceland and norway', 'rollo', '10th', 'the normans', 'normandy', 'west francia', 'first half of the 10th century', 'william the conqueror']


In [18]:
def normalize_text(s):
    """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
    import string, re

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

In [19]:
bert_em_score = []
for p, true_answer in zip(bert_answer, answer[0:500]):
  bert_em_score.append(max((compute_exact_match(p, a)) for a in true_answer))


In [20]:
bert_f1_score = []
for p, true_answer in zip(bert_answer, answer[0:500]):
  bert_f1_score.append(max((compute_f1(p, a)) for a in true_answer))


In [37]:
#bert_answer

['france',
 '10th and 11th centuries',
 'denmark , iceland and norway',
 'rollo',
 '10th',
 'the normans',
 'normandy',
 'west francia',
 'first half of the 10th century',
 'william the conqueror']

In [ ]:
#bert_f1_score

In [21]:
print('Pretrained Bert accuracy (first 500): ', sum(bert_em_score)/len(bert_em_score))

Pretrained Bert accuracy (first 500):  0.366


In [22]:
print('Pretrained Bert Average F1 (first 500): ', sum(bert_f1_score)/len(bert_f1_score))

Pretrained Bert Average F1 (first 500):  0.4070569264069264


ELECTRA


In [23]:
from transformers import ElectraTokenizer, ElectraForQuestionAnswering


In [24]:
electra_tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-discriminator')
electra_model = ElectraForQuestionAnswering.from_pretrained('google/electra-small-discriminator')


Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForQuestionAnswering: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['qa_outputs.weight', 'qa_output

In [25]:
electra_answer = get_answers(context[0:500], question[0:500], electra_model, electra_tokenizer)

In [51]:
#import pickle

#with open('electra_answer_pickle', 'wb') as ep:
#    pickle.dump(electra_answer, ep)

In [ ]:
#with open ('electra_answer_pickle', 'rb') as ep:
#    electra_answer = pickle.load(ep)

In [26]:
electra_em_score = []
for p, true_answer in zip(electra_answer, answer[0:500]):
  electra_em_score.append(max((compute_exact_match(p, a)) for a in true_answer))

In [27]:
electra_f1_score = []
for p, true_answer in zip(electra_answer, answer[0:500]):
  electra_f1_score.append(max((compute_f1(p, a)) for a in true_answer))

In [28]:
print('Pretrained ELECTRA accuracy (first 500): ', sum(electra_em_score)/len(electra_em_score))

Pretrained ELECTRA accuracy (first 500):  0.312


In [29]:
print('Pretrained ELECTRA Average F1 (first 500): ', sum(electra_f1_score)/len(electra_f1_score))

Pretrained ELECTRA Average F1 (first 500):  0.32488260398305063
